# <font color="#418FDE" size="6.5" uppercase>**Modules and Layers**</font>

>Last update: 20260129.
    
By the end of this Lecture, you will be able to:
- Define custom neural network components by subclassing nn.Module and implementing forward methods. 
- Use common nn layers such as Linear, Conv2d, and Dropout to assemble simple models. 
- Inspect and manage model parameters, including initialization and parameter counting. 


## **1. Building Custom Modules**

### **1.1. Creating Custom Modules**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_02/Lecture_B/image_01_01.jpg?v=1769698282" width="250">



>* Custom modules group related operations and parameters
>* They simplify complex models, improving clarity and reuse

>* Decide module behavior, inputs, and outputs clearly
>* Hide internal steps; keep interface stable, flexible

>* Design modules that match your problem domain
>* Swap high-level components to experiment and reuse



In [ ]:
#@title Python Code - Creating Custom Modules

# This script shows creating simple custom modules.
# We use TensorFlow Keras to define custom layers.
# Focus on subclassing Layer and implementing call.

# !pip install tensorflow.

# Import required TensorFlow modules.
import tensorflow as tf

# Set deterministic random seeds for reproducibility.
tf.random.set_seed(42)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Define a simple custom dense like layer.
class MyDenseLayer(tf.keras.layers.Layer):

    # Initialize layer with output units argument.
    def __init__(self, units):
        super().__init__()
        self.units = units

    # Build method creates weights once input shape known.
    def build(self, input_shape):
        last_dim = int(input_shape[-1])
        assert last_dim > 0
        self.w = self.add_weight(
            shape=(last_dim, self.units),
            initializer="glorot_uniform",
            trainable=True,
            name="kernel",
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
            name="bias",
        )

    # Call method defines forward computation logic.
    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)

# Create a small model using the custom layer.
inputs = tf.keras.Input(shape=(4,))

# Apply custom dense layer to the inputs.
x = MyDenseLayer(units=3)(inputs)

# Add a final built in dense output layer.
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

# Build the Keras model object.
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Show a short model summary to inspect shapes.
model.summary()

# Create a tiny batch of example input data.
example_batch = tf.constant([[1.0, 2.0, 3.0, 4.0]])

# Validate input shape before running the model.
assert example_batch.shape == (1, 4)

# Run a forward pass through the model.
output_batch = model(example_batch)

# Print the model output tensor values.
print("Model output:", output_batch.numpy())

# Print the number of trainable parameters.
print("Trainable parameters:", model.count_params())




### **1.2. Init and Forward Methods**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_02/Lecture_B/image_01_02.jpg?v=1769698346" width="250">



>* Initializer defines module structure and components
>* Forward method specifies data flow and computation

>* Initializer builds and connects the module’s parts
>* Forward defines how inputs move through those parts

>* Init defines complex model parts and structure
>* Forward controls data flow, improving reuse and debugging



In [ ]:
#@title Python Code - Init and Forward Methods

# This script shows custom module initialization and forward computation.
# We use TensorFlow Keras layers to mimic PyTorch style modules.
# Focus on __init__ like setup and forward style call method.
# !pip install tensorflow.
# Import required TensorFlow modules.

import tensorflow as tf

# Set a deterministic random seed.
tf.random.set_seed(42)

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Define a simple custom dense block class.
class SimpleDenseBlock(tf.keras.layers.Layer):

    # Initialize internal layers and configuration.
    def __init__(self, units, dropout_rate=0.0):
        super().__init__()
        self.units = int(units)
        self.dropout_rate = float(dropout_rate)

        # Create a dense layer for linear transformation.
        self.dense = tf.keras.layers.Dense(
            units=self.units,
            activation="relu",
        )

        # Create a dropout layer for regularization.
        self.dropout = tf.keras.layers.Dropout(
            rate=self.dropout_rate,
        )

    # Define the forward computation for one input batch.
    def call(self, inputs, training=False):
        dense_out = self.dense(inputs)
        outputs = self.dropout(dense_out, training=training)
        return outputs


# Create a tiny model using the custom block.
class TinyModel(tf.keras.Model):

    # Initialize submodules inside the model.
    def __init__(self):
        super().__init__()
        self.block1 = SimpleDenseBlock(units=8, dropout_rate=0.1)
        self.output_layer = tf.keras.layers.Dense(
            units=1,
            activation="sigmoid",
        )

    # Define how data flows through the model.
    def call(self, inputs, training=False):
        x = self.block1(inputs, training=training)
        outputs = self.output_layer(x)
        return outputs


# Create a small batch of dummy input data.
inputs = tf.random.uniform(shape=(4, 5), minval=-1.0, maxval=1.0)

# Validate the input shape before using the model.
if inputs.shape[1] != 5:
    raise ValueError("Expected input features equal to five.")

# Instantiate the tiny model.
model = TinyModel()

# Run a forward pass to build the model.
outputs = model(inputs, training=False)

# Print input and output shapes to inspect behavior.
print("Input shape:", inputs.shape)
print("Output shape:", outputs.shape)

# Count and print the total number of trainable parameters.
trainable_count = sum(
    int(tf.size(v)) for v in model.trainable_variables
)
print("Trainable parameters:", int(trainable_count))

# Show names of internal variables for clarity.
for var in model.trainable_variables:
    print("Variable:", var.name, "shape:", var.shape)




### **1.3. Mastering super in Modules**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_02/Lecture_B/image_01_03.jpg?v=1769698412" width="250">



>* Calling super activates nn.Module’s built-in features
>* It prepares infrastructure so custom layers work seamlessly

>* Overriding forward customizes computation within framework rules
>* Sometimes reuse parent forward, then add extra steps

>* Calling parent classes prevents subtle training bugs
>* Consistent super use keeps models compatible and reliable



In [ ]:
#@title Python Code - Mastering super in Modules

# This script shows mastering super in modules.
# We use TensorFlow to mimic PyTorch style modules.
# Focus on subclassing and calling parent initializers.

# !pip install tensorflow-2.20.0.

# Import required TensorFlow module.
import tensorflow as tf

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Set deterministic random seed value.
tf.random.set_seed(42)

# Define a simple custom dense like layer.
class MyDense(tf.keras.layers.Layer):

    # Initialize layer and call parent initializer.
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.units = int(units)

    # Build weights once input shape is known.
    def build(self, input_shape):
        last_dim = int(input_shape[-1])
        assert last_dim > 0, "Input features must be positive"
        self.w = self.add_weight(
            shape=(last_dim, self.units), initializer="glorot_uniform"
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="zeros"
        )
        super().build(input_shape)

    # Define forward computation for the layer.
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

# Define a custom module that reuses MyDense.
class MyBlock(tf.keras.layers.Layer):

    # Call parent initializer before creating sublayers.
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.base = MyDense(units)
        self.extra_bias = self.add_weight(
            shape=(units,), initializer="zeros"
        )

    # Forward pass extends base layer behavior.
    def call(self, inputs):
        base_out = self.base(inputs)
        return base_out + self.extra_bias

# Create a tiny input batch tensor.
inputs = tf.constant([[1.0, 2.0], [3.0, 4.0]])

# Instantiate custom block with specific units.
block = MyBlock(units=3, name="my_block")

# Run a forward pass to build weights.
outputs = block(inputs)

# Print input and output shapes clearly.
print("Input shape:", inputs.shape)
print("Output shape:", outputs.shape)

# Show that variables were registered correctly.
print("Number of trainable variables:", len(block.trainable_variables))

# Print each variable name and shape briefly.
for var in block.trainable_variables:
    print("Var:", var.name, "Shape:", var.shape)

# Confirm that calling block again reuses same weights.
print("Second call output equal:", tf.reduce_all(block(inputs) == outputs).numpy())




## **2. Core Neural Layers**

### **2.1. Linear Layers and Activations**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_02/Lecture_B/image_02_01.jpg?v=1769698478" width="250">



>* Linear layers turn input features into outputs
>* Stacked layers build increasingly abstract learned representations

>* Linear layers alone stay a simple transformation
>* Nonlinear activations give networks rich expressive power

>* Alternate linear layers and activations to build features
>* Layer sizes and activations adapt models to tasks



In [ ]:
#@title Python Code - Linear Layers and Activations

# This script shows linear layers and activations.
# It uses TensorFlow dense layers for clarity.
# Run cells to see shapes and simple outputs.

# !pip install tensorflow.

# Import TensorFlow and NumPy libraries.
import tensorflow as tf
import numpy as np

# Set deterministic random seeds for reproducibility.
tf.random.set_seed(42)
np.random.seed(42)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Create a small batch of input feature vectors.
inputs = np.random.randn(4, 3).astype("float32")

# Show the input shape and a small preview.
print("Input shape:", inputs.shape)
print("First input row:", inputs[0])

# Define a simple model with two dense layers.
model = tf.keras.Sequential([
    tf.keras.layers.Dense(5, activation=None, input_shape=(3,)),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(2, activation="sigmoid"),
])

# Run a forward pass to get model outputs.
outputs = model(inputs)

# Show the output shape and a small preview.
print("Output shape:", outputs.shape)
print("First output row:", outputs[0].numpy())

# Access the first dense layer from the model.
first_dense = model.layers[0]

# Get weights and biases from the first dense layer.
weights, biases = first_dense.get_weights()

# Print shapes of weights and biases for inspection.
print("First layer weights shape:", weights.shape)
print("First layer biases shape:", biases.shape)

# Count total trainable parameters in the model.
model_params = model.count_params()

# Print the total number of trainable parameters.
print("Total trainable parameters:", model_params)




### **2.2. Convolutions and Pooling**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_02/Lecture_B/image_02_02.jpg?v=1769698535" width="250">



>* Convolutional layers slide shared filters over structured data
>* Stacked convolutions build hierarchical features with fewer parameters

>* Pooling summarizes local regions, reducing feature map size
>* It adds translation invariance and saves computation

>* Stack conv and pooling blocks to extract features
>* Flatten features, use linear layers for tasks



In [ ]:
#@title Python Code - Convolutions and Pooling

# This script shows basic convolutions and pooling.
# It uses TensorFlow to build tiny models.
# Focus on images with simple feature maps.

# !pip install tensorflow.

# Import required standard libraries.
import os
import random
import numpy as np

# Import TensorFlow and Keras layers.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set deterministic random seeds.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Load MNIST dataset from Keras.
(x_train, y_train), _ = keras.datasets.mnist.load_data()

# Select a small subset for speed.
num_samples = 64
x_small = x_train[:num_samples]
y_small = y_train[:num_samples]

# Normalize pixel values to range zero one.
x_small = x_small.astype("float32") / 255.0

# Add channel dimension for convolutions.
x_small = np.expand_dims(x_small, axis=-1)

# Validate input shape before modeling.
assert x_small.shape[1:] == (28, 28, 1)

# Build a simple conv and pooling model.
model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(filters=4, kernel_size=3, activation="relu"),
    layers.MaxPooling2D(pool_size=2),
    layers.Conv2D(filters=8, kernel_size=3, activation="relu"),
    layers.AveragePooling2D(pool_size=2),
    layers.Flatten(),
    layers.Dense(10, activation="softmax"),
])

# Show model summary in one line.
model.summary(print_fn=lambda x: None)

# Count total trainable parameters.
trainable_params = np.sum([
    np.prod(v.shape) for v in model.trainable_weights
])

# Compile model with simple settings.
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Train briefly on the tiny subset.
history = model.fit(
    x_small,
    y_small,
    epochs=2,
    batch_size=16,
    verbose=0,
)

# Evaluate on the same subset silently.
loss, acc = model.evaluate(
    x_small,
    y_small,
    verbose=0,
)

# Get one example and its prediction.
example = x_small[0:1]
true_label = int(y_small[0])
probs = model.predict(example, verbose=0)[0]

# Find predicted class index.
pred_label = int(np.argmax(probs))

# Print key results in few lines.
print("Trainable parameters:", int(trainable_params))
print("Subset accuracy:", round(float(acc), 3))
print("True label:", true_label, "Predicted:", pred_label)




### **2.3. Regularization with Dropout BatchNorm**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_02/Lecture_B/image_02_03.jpg?v=1769698604" width="250">



>* Dropout randomly disables units to reduce overfitting
>* BatchNorm stabilizes activations, speeding deeper network training

>* Combine Conv, BatchNorm, activations, and dropout layers
>* They stabilize training and improve generalization on data

>* Dropout and BatchNorm act differently during inference
>* Correct modes give stable, reliable real-world predictions



In [ ]:
#@title Python Code - Regularization with Dropout BatchNorm

# This script shows dropout and batch normalization.
# We build a tiny image model using TensorFlow layers.
# Focus on regularization behavior during training evaluation.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import random
import numpy as np

# Import TensorFlow and Keras layers.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set deterministic random seeds.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Load MNIST dataset from Keras.
(x_train, y_train), _ = keras.datasets.mnist.load_data()

# Select a small subset for speed.
num_samples = 512
x_train = x_train[:num_samples]
y_train = y_train[:num_samples]

# Normalize pixel values to range zero one.
x_train = x_train.astype("float32") / 255.0

# Add channel dimension for Conv2D.
x_train = np.expand_dims(x_train, axis=-1)

# Validate input shape dimensions.
assert x_train.shape[1:] == (28, 28, 1)

# Build a small model with Conv2D.
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(16, (3, 3), activation="relu")(inputs)
x = layers.BatchNormalization()(x)

# Add dropout after convolution block.
x = layers.Dropout(0.3)(x)

# Flatten and add dense layer.
x = layers.Flatten()(x)
x = layers.Dense(32, activation="relu")(x)

# Add another dropout for regularization.
x = layers.Dropout(0.5)(x)

# Output layer for ten digit classes.
outputs = layers.Dense(10, activation="softmax")(x)

# Create the Keras model object.
model = keras.Model(inputs=inputs, outputs=outputs)

# Compile model with simple optimizer.
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Show model summary in one short line.
print("Model has", model.count_params(), "trainable parameters.")

# Train briefly with silent verbose setting.
history = model.fit(x_train, y_train, epochs=2, batch_size=64, verbose=0)

# Take a small batch for demonstration.
x_batch = x_train[:4]

# Get predictions in training mode.
train_preds = model(x_batch, training=True)

# Get predictions in inference mode.
eval_preds = model(x_batch, training=False)

# Convert predictions to numpy arrays.
train_preds_np = train_preds.numpy()
eval_preds_np = eval_preds.numpy()

# Print shapes to confirm behavior.
print("Batch shape:", x_batch.shape)
print("Train preds shape:", train_preds_np.shape)
print("Eval preds shape:", eval_preds_np.shape)

# Compute mean difference between predictions.
diff = np.mean(np.abs(train_preds_np - eval_preds_np))

# Print explanation of dropout effect.
print("Mean abs difference train vs eval:", float(diff))
print("Larger difference shows active dropout during training.")
print("BatchNorm uses running statistics during evaluation.")
print("This combination helps regularize the small model.")
print("Script finished successfully with regularization demo.")



## **3. Managing Model Parameters**

### **3.1. Iterating Model Parameters**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_02/Lecture_B/image_03_01.jpg?v=1769698676" width="250">



>* Iterating parameters reveals all learnable weights and biases
>* Gives transparency for debugging, optimization, and design checks

>* See parameter values, names, shapes, trainable flags
>* Group parameters for different training or regularization strategies

>* Use parameter iteration for freezing and custom optimization
>* Compute stats for deployment and precise control



In [ ]:
#@title Python Code - Iterating Model Parameters

# This script shows iterating model parameters.
# We use TensorFlow Keras dense layers.
# Focus is on inspecting trainable weights.

# !pip install tensorflow==2.20.0.

# Import required TensorFlow and NumPy modules.
import tensorflow as tf
import numpy as np

# Set deterministic random seeds for reproducibility.
tf.random.set_seed(42)
np.random.seed(42)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Define a simple sequential model with dense layers.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4,)),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(3, activation="softmax"),
])

# Build the model by calling it on dummy data.
dummy_input = tf.zeros((1, 4), dtype=tf.float32)
_ = model(dummy_input)

# Confirm the model output shape is as expected.
assert _.shape == (1, 3)

# Print a short header for parameter inspection.
print("\nIterating over trainable parameters:")

# Iterate over each trainable variable in the model.
for var in model.trainable_variables:
    name = var.name
    shape = var.shape
    size = np.prod(shape)
    trainable = var.trainable

    # Print a concise summary for each parameter.
    print(
        f"Name: {name}, shape: {shape}, size: {size}, trainable: {trainable}"
    )

# Compute total number of trainable parameters.
param_counts = [int(np.prod(v.shape)) for v in model.trainable_variables]

# Sum parameter counts to get the total size.
total_params = int(np.sum(param_counts))

# Print the final total parameter count clearly.
print("\nTotal trainable parameters in model:", total_params)



### **3.2. Initializing Model Parameters**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_02/Lecture_B/image_03_02.jpg?v=1769698741" width="250">



>* Initialization sets learning’s starting point and stability
>* Choose layer-aware weight scales to keep signals balanced

>* Defaults work, but complex models need control
>* Choose schemes by activation, task, and stability

>* Decide when to reinitialize and reuse weights
>* Use domain knowledge; treat initialization as ongoing design



In [ ]:
#@title Python Code - Initializing Model Parameters

# This script shows simple parameter initialization concepts.
# We use TensorFlow to build tiny dense layers.
# Focus on inspecting and reinitializing layer weights.

# !pip install tensorflow==2.20.0.

# Import required modules from TensorFlow.
import tensorflow as tf

# Set a deterministic random seed value.
tf.random.set_seed(42)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Create a simple dense layer with default initialization.
default_layer = tf.keras.layers.Dense(
    units=3, activation="relu", input_shape=(4,)
)

# Build the layer by calling it on dummy input.
dummy_input = tf.zeros((1, 4))

# Run a forward pass to ensure weights are created.
_ = default_layer(dummy_input)

# Get the weights and biases from the layer.
default_weights, default_biases = default_layer.get_weights()

# Print basic information about default initialized parameters.
print("Default weights shape:", default_weights.shape)

# Show a small sample of default weight values.
print("First row default weights:", default_weights[0])

# Show default bias values for the dense layer.
print("Default biases:", default_biases)

# Now create a new dense layer with custom initializer.
custom_layer = tf.keras.layers.Dense(
    units=3, activation="relu", input_shape=(4,)
)

# Build the custom layer using another dummy input.
_ = custom_layer(dummy_input)

# Manually create new weight and bias tensors.
custom_w = tf.random.normal(shape=(4, 3), stddev=0.05)

# Initialize biases to small positive constant values.
custom_b = tf.ones(shape=(3,)) * 0.1

# Validate shapes before assigning new weights.
if custom_w.shape == custom_layer.kernel.shape:
    custom_layer.set_weights([custom_w.numpy(), custom_b.numpy()])

# Retrieve the updated weights and biases.
new_weights, new_biases = custom_layer.get_weights()

# Print shapes to confirm successful reinitialization.
print("Custom weights shape:", new_weights.shape)

# Show a small sample of custom initialized weights.
print("First row custom weights:", new_weights[0])

# Show custom bias values for the dense layer.
print("Custom biases:", new_biases)



### **3.3. Counting Model Parameters**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_02/Lecture_B/image_03_03.jpg?v=1769698776" width="250">



>* Counting parameters reveals model size and cost
>* Layer design choices directly change total parameters

>* View each layer as weights plus biases
>* Sum layer contributions to compare model designs

>* Parameter limits control memory, speed, and scalability
>* Regular parameter checks balance accuracy and overfitting



In [ ]:
#@title Python Code - Counting Model Parameters

# This script explores counting model parameters.
# It uses TensorFlow dense layers for illustration.
# Focus on understanding shapes and parameter formulas.

# !pip install tensorflow.

# Import TensorFlow and NumPy for this lesson.
import tensorflow as tf
import numpy as np

# Set a deterministic random seed for reproducibility.
tf.random.set_seed(42)
np.random.seed(42)

# Print TensorFlow version in one concise line.
print("TensorFlow version:", tf.__version__)

# Define a simple sequential model with dense layers.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(8,)),
    tf.keras.layers.Dense(4, activation="relu"),
    tf.keras.layers.Dense(3, activation="relu"),
    tf.keras.layers.Dense(1, activation="linear"),
])

# Build the model by calling it on dummy input.
dummy_input = tf.zeros((1, 8), dtype=tf.float32)
_ = model(dummy_input)

# Confirm the dummy input shape is as expected.
assert dummy_input.shape == (1, 8)

# Function to count parameters in a single variable.
def count_params_in_variable(variable):
    shape = variable.shape
    size = np.prod(shape)
    return int(size)

# Function to count parameters for each layer.
def count_params_per_layer(model):
    layer_param_info = []
    for layer in model.layers:
        layer_params = 0
        for var in layer.trainable_variables:
            layer_params += count_params_in_variable(var)
        layer_param_info.append((layer.name, layer_params))
    return layer_param_info

# Get parameter counts for each layer in the model.
layer_info = count_params_per_layer(model)

# Compute the total number of trainable parameters.
trainable_variables = model.trainable_variables
total_params = sum(count_params_in_variable(v) for v in trainable_variables)

# Print a short header for the parameter table.
print("\nLayer name and parameter counts:")

# Loop through layers and print their parameter counts.
for name, params in layer_info:
    print(f"{name:15s} -> {params:4d} params")

# Explain the manual formula for the first dense layer.
input_units = 8
output_units_layer1 = 4
weights_layer1 = input_units * output_units_layer1
biases_layer1 = output_units_layer1

# Compute the manual parameter count for first dense layer.
manual_params_layer1 = weights_layer1 + biases_layer1

# Print manual and model counts for comparison.
print("\nManual params for first dense layer:", manual_params_layer1)

# Print the total number of trainable parameters.
print("Total trainable parameters:", total_params)




# <font color="#418FDE" size="6.5" uppercase>**Modules and Layers**</font>


In this lecture, you learned to:
- Define custom neural network components by subclassing nn.Module and implementing forward methods. 
- Use common nn layers such as Linear, Conv2d, and Dropout to assemble simple models. 
- Inspect and manage model parameters, including initialization and parameter counting. 

In the next Module (Module 3), we will go over 'Training Workflow'